- .: A dot represents a match to the reference base on the forward strand.
- ,: A comma represents a match to the reference base on the reverse strand.
- A, C, G, T, N: These letters represent a mismatched base (SNP) on the forward strand.
- a, c, g, t, n: These letters represent a mismatched base (SNP) on the reverse strand. 

In [27]:
import pandas as pd 
import numpy as np
import re

TISSUE="Dura"
#SAMPLES = ["220930", "221026", "221202", "230127", "230323", "230405", "230419", "230822", "230920"]
SAMPLES = [ "230526" ]
INTERVAL = pd.read_csv( "/data/project/Meningioma/03.mpileup/hg38/NF2.bed", sep = '\t', header = None ) 
DEPTH_THRESHOLD = 20

df = pd.DataFrame (  index = None,  columns = ["chr", "pos", "REF", "ALT", "depth",  "ALT_read", "readinfo", "baseinfo"] )
#df.index = SAMPLES

variant_list = []

for date in SAMPLES :
    OUTPUT_PATH="/data/project/Meningioma/03.mpileup/hg38/" + TISSUE + "/01.mpileup/" + date + "_WES.pileup"

    input_df = pd.read_csv ( OUTPUT_PATH, sep = "\t", header = None, names = ["chr", "pos", "REF", "depth", "readinfo", "baseinfo"] )
    variant_list = input_df.iloc[:, 0].astype(str) + ":" + input_df.iloc[:, 1].astype(str)    # + " " + input_df.iloc[:, 2].astype(str) 

    for k in range (input_df.shape[0]):
        st =  (input_df.iloc [k]["readinfo"])
        snv_matches = re.compile("[AGCTagct]").findall( st )
        indel_matches = re.compile("[+-]").findall( st )

        #print (snv_matches, indel_matches, st )

        # print (date , k,  snv_matches, st )
        if (  (len (snv_matches) >= 1) | ( len(indel_matches) >= 1) ) & ( int ((input_df.iloc [k]["depth"])) >= DEPTH_THRESHOLD ):
            if len (snv_matches) >= 1:
                variant_list[k] = variant_list[k] + ">" + str ( snv_matches[0] ).upper()
                add_matrix = input_df.iloc[k][ [0, 1, 2] ].tolist() + [ str ( snv_matches[0] ).upper() ] + [input_df.iloc[k][ 3 ].tolist()] + [ len (snv_matches) ]  + input_df.iloc[k][-2:].tolist() 
            if len (indel_matches) >= 1:
                add_matrix = input_df.iloc[k][ [0, 1, 2] ].tolist() + [ str ( indel_matches[0] ).upper() ] + [input_df.iloc[k][ 3 ].tolist()] + [ len (indel_matches) ]  + input_df.iloc[k][-2:].tolist() 
            
            df.loc[len(df.index)] = add_matrix

# df.columns = list ( variant_list )
# df = df [ list ( INTERVAL.iloc[:,0].astype (str) + ":" + INTERVAL.iloc[:,2].astype (str) )]

df

['chr22', 29603795, 'C', 23, 'A.AAA.AaAAAAAAA.A.AA.A.', 'HHRHHHHGHGGHHHHGGGGGGGH']
['chr22', 29603806, 'A', 37, '........,........................G..^].', 'InHoIIiI.I=IIIIIIHIHIHIHHHHHHHGGG2EEC']
['chr22', 29603889, 'G', 81, 'CcCcCCCCCCCCCCCCcCCCCCCCCCCCCCCCCCCcCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCcC', '9FHGHoHHHHHHGHHGGHGHHHHHHHHHHHH@HHHHHGHHGHGHGHGGGGGHGHGGGGGGGGGGGGGGGGGGGFGGFFGH9']
['chr22', 29603905, 'G', 95, '.$.$.,..................,..T..........,..............................,...........................', 'AAHDIHHHHHHHHHHHHHHHHHEHH.HHHHHGHHGGFGGHGHGHGGHGGHGGGGHHHGHGGGGAGGHFGGGGGGGGGGGGGGFFGFGFGFGHFGG']
['chr22', 29603942, 'A', 141, '.C..,............,.............................,..............................................................................,.........^].^].^].^].^].', 'E.:GGFFFFFFEFFnEEHEF7FFFFFFE7F8E8FF::`FFFFFFFFFI:FFFFEEFFFFFFFFF8FEFFEFEFFEEFFEEEFEEEF;EEEEEEFEEEE9E9EEEEEEEEE;EEEEEEED;DEDDEDJE;9CDDCCDAAAAA']
['chr22', 29603952, 'G', 171, '.$..........,..

,chr,pos,REF,ALT,depth,ALT_read,readinfo,baseinfo
0,chr22,29603795,C,A,23,17,A.AAA.AaAAAAAAA.A.AA.A.,HHRHHHHGHGGHHHHGGGGGGGH
1,chr22,29603806,A,G,37,1,"........,........................G..^].",InHoIIiI.I=IIIIIIHIHIHIHHHHHHHGGG2EEC
2,chr22,29603889,G,C,81,81,CcCcCCCCCCCCCCCCcCCCCCCCCCCCCCCCCCCcCCCCCCCCCC...,9FHGHoHHHHHHGHHGGHGHHHHHHHHHHHH@HHHHHGHHGHGHGH...
3,chr22,29603905,G,T,95,1,".$.$.,..................,..T..........,..........",AAHDIHHHHHHHHHHHHHHHHHEHH.HHHHHGHHGGFGGHGHGHGG...
4,chr22,29603942,A,C,141,1,".C..,............,...............................",E.:GGFFFFFFEFFnEEHEF7FFFFFFE7F8E8FF::`FFFFFFFF...
...,...,...,...,...,...,...,...,...
712,chr22,29694930,C,G,110,1,".,,,,,,,,,.,,,,,,,.,,,,,,,,,,,,,,,,,,,.,,g,,,,...",GGGGFGG@FGnGGGGGGGnGGGGGGGGGGGHHGGGGGGGHH/HGHG...
713,chr22,29694973,T,A,68,1,",$,$,,,,,,,,,,,,.,,,,,a,..,,,.,,,,,,,,.,,,,,,,...",BBFHFFGFFGGGGGI<GGGGI>HIHGHHHHG>H=HGI=H;HHHHHH...
714,chr22,29694988,T,G,54,1,",$,,,,,..,,,.,,,,,,,,.,,,g,,,,,,.,,,,.,,.,,.,,...",C9EEEEGGEEEGEEE;EEFEGFFE.EEEEFEGF:FFAFFFFEGFFF...
715,chr22,29694996,T,G,53,1,",$..,,,.,,,,,,,,.,,,,,,,,,,.,,,,.,,,.,,.,g,.,,...",@IIHGHIGGGGGG?<HHGGGGGGGGGIGHHGpHH<IHHIH.HHGHI...


In [34]:
df [ df["ALT_read"] >= 3 ]

,chr,pos,REF,ALT,depth,ALT_read,readinfo,baseinfo
0,chr22,29603795,C,A,23,17,A.AAA.AaAAAAAAA.A.AA.A.,HHRHHHHGHGGHHHHGGGGGGGH
2,chr22,29603889,G,C,81,81,CcCcCCCCCCCCCCCCcCCCCCCCCCCCCCCCCCCcCCCCCCCCCC...,9FHGHoHHHHHHGHHGGHGHHHHHHHHHHHH@HHHHHGHHGHGHGH...
120,chr22,29637047,T,A,28,14,".,,aa.a,,,a,,,Aaaaaaaa,,aa,,",IEGFEIFGGGFGGGFGGGG?GGGHGGHH
130,chr22,29639005,G,T,119,63,.TT.T.T.TTT..TT..TTTtT..TTT.T.TTTT..T..TT.T.T....,/I?IIIIIIII?III.HHHIGIIII@@IIII@I@IIIBIIIIIIII...
165,chr22,29642163,A,C,243,121,".CCCC..CCCCC.CC..CC.CcC.CCCC..CCC.CC.,..CCC.C....",<CaFGHHGGnnGHGnmGGGHGGGnGmGmGHGGGGFmHFHHGGGH:H...
204,chr22,29655444,C,T,53,16,",TT,T..TT.....T..T..T.T.,.T...T..T..T.....T.,T...",:mG4G:FGFFF;FFGFFGFFFFGF:FFFFFFFEFEEFEEEEEFE;F...
233,chr22,29655689,C,A,241,3,".$.$.$,$,$.,.......,,,,.,,...,,.,,,,,...,,,,,....",TSEECoF?oTHHgUHIHIHHGG>VHGGGGGHHGUGGHHHHn<eoHH...
236,chr22,29655702,C,A,205,15,".$,$,$,$...,,,,,.,,,,,.,,,,...,,,..,,..,,,,,.....",SEECFFmGGHHGTHIHBIgHIHHFUfGHHmTHHneGGGBHFjHGHH...
240,chr22,29655711,A,T,140,5,",$,$,$,$,$,$..,,,,.,,.,,,,,,.,,,,.,,,,,.,,,,,....",;CCCCEFlEDDEdF?8AGFFFGGFFFFFFEFFERF@FFEDFFFFSF...
242,chr22,29655715,G,-,160,3,".$,$,$,$,.,,,,,.,,,,,.,,,,.,,,,,,,,..,,,,,,,,,...",@@@@FHGF?GFoHGHHGSFFGFoFGFFFFFGHnGGGGGGGGGGGGF...
